# Batch Process Source HEC-RAS Models
Generate FIM Libraries for multiple HEC-RAS Models

In [1]:
%load_ext autoreload
%autoreload 2

from scripts.processing import *
from scripts.setup import get_models_from_stac
import os

### `Parameters`

In [2]:
stac_endpoint = "https://stac2.dewberryanalytics.com"  # STAC API endpoint
stac_collection_id = "ble_12040103_EastForkSanJacinto"  # Collection name

root_dir = r"Z:\collections\ble_12040103_EastForkSanJacinto"

stop_on_error = False

db_path = os.path.join(root_dir, "ripple.gpkg")
source_models_dir = os.path.join(root_dir, "source_models")
submodels_dir = os.path.join(root_dir, "submodels")
library_dir = os.path.join(root_dir, "library")

### Get Working Models

In [ ]:
models_data = get_models_from_stac(stac_endpoint, stac_collection_id)
model_ids = list(models_data.keys())

### Run `conflate_model`

In [ ]:
succeded_models, failed_models, not_accepted_models = execute_model_step(model_ids, "conflate_model", db_path, source_models_dir)
if stop_on_error and (len(failed_models) + len(not_accepted_models)) > 0:
    raise Exception("One or more model failed. Stopping Execution. Please address and then run below cells.")

### Load Conflation Information to Database

In [ ]:
load_conflation(model_ids, source_models_dir, db_path)

### Update `network_to_id` Table in Database
Build a modified network by skipping over reaches eclipsed by adjacent reaches

In [ ]:
update_network(db_path)

### Get Working Reaches

In [ ]:
reaches = get_reaches_by_models(db_path, model_ids)

### Run `extract_submodel`

In [ ]:
succeded_reaches, failed_reaches, not_accepted_reaches = execute_step(reaches, "extract_submodel", db_path, source_models_dir, submodels_dir)
if stop_on_error and (len(failed_reaches) + len(not_accepted_reaches)) > 0:
    raise Exception("One or more reach failed. Stopping Execution. Please address and then run below cells.")

### Run `create_ras_terrain`

In [ ]:
succeded_reaches, failed_reaches, not_accepted_reaches = execute_step(succeded_reaches, "create_ras_terrain", db_path, source_models_dir, submodels_dir)
if stop_on_error and (len(failed_reaches) + len(not_accepted_reaches)) > 0:
    raise Exception("One or more reach failed. Stopping Execution. Please address and then run below cells.")

### Run `create_model_run_normal_depth`

In [ ]:
succeded_reaches, failed_reaches, not_accepted_reaches = execute_step(succeded_reaches, "create_model_run_normal_depth", db_path, source_models_dir, submodels_dir)
if stop_on_error and (len(failed_reaches) + len(not_accepted_reaches)) > 0:
    raise Exception("One or more reach failed. Stopping Execution. Please address and then run below cells.")

### Run `run_incremental_normal_depth`

In [ ]:
succeded_reaches, failed_reaches, not_accepted_reaches = execute_step(succeded_reaches, "run_incremental_normal_depth", db_path, source_models_dir, submodels_dir)
if stop_on_error and (len(failed_reaches) + len(not_accepted_reaches)) > 0:
    raise Exception("One or more reach failed. Stopping Execution. Please address and then run below cells.")

### Run `run_known_wse` for Network

In [ ]:
execute_kwse_for_network([(reach[0], None) for reach in reaches if reach[1] is None], submodels_dir, db_path, False, library_dir, False)

## Merge Rating Curves

In [ ]:
load_all_rating_curves(library_dir, db_path)

In [ ]:
# !jupyter nbconvert --to html "process copy 2.ipynb"
